# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv file and read as dataframe
City_Data_file_path = "../WeatherPy/Weather_Output/City_Data.csv"
City_Data = pd.read_csv(City_Data_file_path)

# review new dataframe
City_Data.describe()

,Lat,Lng,Max Temp °F,Humidity,Cloudiness,Wind Speed,Date
count,562.000000,562.000000,562.000000,562.000000,562.000000,562.000000,5.620000e+02
mean,22.328345,14.319680,58.249199,74.391459,55.749110,7.750730,1.603598e+09
std,33.088482,89.414048,20.852194,20.072413,38.797836,5.498242,1.242100e+02
min,-54.800000,-179.170000,3.990000,13.000000,0.000000,0.090000,1.603597e+09
25%,-6.287500,-65.050000,43.307500,63.250000,18.000000,3.360000,1.603598e+09
50%,28.580000,19.100000,62.575000,79.000000,69.000000,6.555000,1.603598e+09
75%,50.327500,91.285000,76.367500,90.000000,93.750000,10.290000,1.603598e+09
max,78.220000,179.320000,96.800000,100.000000,100.000000,27.940000,1.603598e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Set variants for Lat and Lng as locations and Humidity as the weight
locations = City_Data[['Lat','Lng']]
humidity_weight = City_Data['Humidity']
fig = gmaps.figure()

# Add Heatmap layer to map
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity_weight)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_data_df = City_Data.dropna(how="any")
city_data_df.describe()

,Lat,Lng,Max Temp °F,Humidity,Cloudiness,Wind Speed,Date
count,554.000000,554.000000,554.000000,554.000000,554.000000,554.000000,5.540000e+02
mean,22.981823,14.296245,58.178321,74.709386,56.138989,7.753538,1.603598e+09
std,32.870286,90.057980,20.990105,19.807035,38.793894,5.505723,1.240755e+02
min,-54.800000,-179.170000,3.990000,13.000000,0.000000,0.090000,1.603597e+09
25%,-4.307500,-67.452500,43.067500,64.250000,19.250000,3.360000,1.603598e+09
50%,29.085000,20.920000,62.405000,79.500000,70.000000,6.555000,1.603598e+09
75%,50.695000,93.275000,76.727500,90.000000,94.000000,10.290000,1.603598e+09
max,78.220000,179.320000,96.800000,100.000000,100.000000,27.940000,1.603598e+09


In [6]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
#     A max temperature lower than 80 degrees but higher than 70
#     Wind speed less than 10 mph
#     Zero cloudiness

ideal_city = city_data_df.loc[(city_data_df['Max Temp °F']<80) & (city_data_df['Max Temp °F']>70) &
                               (city_data_df['Wind Speed']<10) & (city_data_df['Cloudiness']== 0)]
# Display dataframe
ideal_city

,City,Lat,Lng,Max Temp °F,Humidity,Cloudiness,Wind Speed,Country,Date
30,Uruguaiana,-29.75,-57.09,73.4,78,0,5.82,BR,1603597480
65,Mandiana,10.63,-8.68,71.6,57,0,2.17,GN,1603597728
137,Chabahar,25.29,60.64,71.6,26,0,6.93,IR,1603597756
214,Saint-Pierre,-21.34,55.48,77.0,47,0,5.82,RE,1603597787
272,Saint-Philippe,-21.36,55.77,77.0,47,0,5.82,RE,1603597810
307,Bandar-e Lengeh,26.56,54.88,78.8,54,0,9.17,IR,1603597824


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create hotel_df to be updated with Hotel Names
hotel_df = ideal_city

# Setup For Loop
for index, row in hotel_df.iterrows():
    coordinates = f"{row['Lat']}, {row['Lng']}"

# Error Handler
    try:
        for hotel in ideal_city:
            params = {
                "location": coordinates,
                "type": "lodging",
                "radius": 5000,
                "key": g_key
            }
# Base URL
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using params dictionary
        response = requests.get(base_url, params=params).json()

# extract results
        results = response['results']
        ihotel_name_val = results[0]["name"]

# Assign Hotel Name (if available) to hotel_df
        hotel_df.at[index,'Hotel Name'] = ihotel_name_val
    except:
        hotel_df.at[index,'Hotel Name'] = "Hotel Not Found"

In [10]:
# Display updated dataframe with 'Hotel Name'
hotel_df

,City,Lat,Lng,Max Temp °F,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
30,Uruguaiana,-29.75,-57.09,73.4,78,0,5.82,BR,1603597480,Fares Turis Hotel
65,Mandiana,10.63,-8.68,71.6,57,0,2.17,GN,1603597728,La maison De Metre Malick
137,Chabahar,25.29,60.64,71.6,26,0,6.93,IR,1603597756,هتل گدروشیا
214,Saint-Pierre,-21.34,55.48,77.0,47,0,5.82,RE,1603597787,Lindsey Hôtel
272,Saint-Philippe,-21.36,55.77,77.0,47,0,5.82,RE,1603597810,"Chambres d'hôte ""La Trinité"""
307,Bandar-e Lengeh,26.56,54.88,78.8,54,0,9.17,IR,1603597824,کافی نت پارس


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))